MinHash Signatures Computation

In [9]:
import random
import json

def minhash_signature(node, num_permutations):
    # Initialize the hash function parameters
    a = [random.randint(1, 2**32 - 1) for i in range(num_permutations)]
    b = [random.randint(1, 2**32 - 1) for i in range(num_permutations)]
    p = 4294967311  # A prime number larger than 2^32 - 1
    
    # Compute the MinHash signature for the node
    signature = []
    for i in range(num_permutations):
        min_hash = float('inf')
        for neighbor in G.neighbors(node):
            hash_val = (a[i] * neighbor + b[i]) % p
            if hash_val < min_hash:
                min_hash = hash_val
        signature.append(min_hash)
        
    return signature[:100]  # Return only the first 100 values

# Load the email-Eu-core network using networkx
import networkx as nx
G = nx.read_edgelist("D:\MTech\SEMESTER 2\IT752 Web and Social Computing\WSC Project\Dataset\email-Eu-core\email-Eu-core.txt", nodetype=int)

# Compute the MinHash signatures for all nodes in the network
num_permutations = 5000
minhash_signatures = {}
for node in G.nodes():
    signature = minhash_signature(node, num_permutations)
    minhash_signatures[node] = signature

with open('minhash_signatures.json', 'w') as f:
    json.dump(minhash_signatures, f)

for x in minhash_signatures:
    print("Signature for " + str(x) + ":-> ")
    print(minhash_signatures[x])


Signature for 0:-> 
[42833148, 133722008, 298002219, 50108564, 138438884, 224181752, 132842046, 91290111, 45113255, 4144593, 112748325, 19674591, 83697515, 89920102, 32276572, 34208767, 70817194, 240371294, 92493016, 38604420, 70716926, 90477460, 11507531, 185870473, 39489297, 142475646, 40537639, 192984371, 32010073, 16969907, 64093057, 46043115, 143499642, 121305022, 12712064, 83651236, 87998180, 15208520, 4544094, 213866020, 9268419, 381596237, 91901827, 1351007, 326795783, 146587344, 136338064, 14733062, 38270887, 51982640, 33735844, 4991649, 271678367, 8051592, 37527282, 191678104, 211623216, 54321002, 85033446, 26807891, 43339219, 202764292, 15460941, 162296925, 56472151, 122527123, 291108300, 9008644, 44304856, 123398358, 9307309, 17664810, 65650890, 116906727, 16344052, 87538964, 177370117, 44713041, 15479583, 139968199, 175762353, 53771337, 20110980, 344927767, 125430240, 135696076, 317364712, 27644306, 157556001, 20417529, 43000471, 138008433, 70705497, 9745526, 31838828, 434

LSH computation

In [10]:
from itertools import combinations

def lsh(minhash_signatures, num_bands, band_size):
    # Initialize the LSH buckets
    buckets = {}
    for i in range(num_bands):
        buckets[i] = {}
    
    # Perform LSH on the MinHash signatures
    for node, signature in minhash_signatures.items():
        for i in range(num_bands):
            band = signature[i*band_size:(i+1)*band_size]
            band_str = str(band)
            if band_str not in buckets[i]:
                buckets[i][band_str] = []
            buckets[i][band_str].append(node)
            
    # Find candidate pairs from the LSH buckets
    candidate_pairs = set()
    for i in range(num_bands):
        for bucket in buckets[i].values():
            if len(bucket) > 1:
                for pair in combinations(bucket, 2):
                    candidate_pairs.add(pair)
                    
    return candidate_pairs

# Load the MinHash signatures computed in the previous example
import json
with open("minhash_signatures.json", "r") as f:
    minhash_signatures = json.load(f)

# Perform LSH with 20 bands of size 5 each
num_bands = 2500
band_size = 2
candidate_pairs = lsh(minhash_signatures, num_bands, band_size)

print("Number of candidate pairs:", len(candidate_pairs))


Number of candidate pairs: 504510
